## AIHub Json Preprocessing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [ ]:
%pip install ray

In [ ]:
import re
import os
import kss
import ray
import json
import time
import inspect
from time import sleep
from tqdm import tqdm
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [ ]:
pwd

### AIHUB 대규모 웹데이터 기반 한국어 말뭉치 데이터

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=624)

#### Convert JSON File to TXT File

In [ ]:
from data_preprocessing import make_topic_json_txt_file_path_list
from data_preprocessing import count_number_of_txt_file_with_batch_list
from data_preprocessing import write_jsontext_to_txt_file_with_batch_list

In [ ]:
json_folder_list = ['AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/',
             'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/']

topic_name_list = ['01_IT_과학', '02_건강', '03_경제', '04_교육', '05_국제', '06_라이프스타일',
                   '07_문화', '08_사건사고', '09_사회일반', '10_산업', '11_스포츠', '12_여성복지',
                   '13_여행레저', '14_연예', '15_정치', '16_지역', '17_취미']

txt_path_list = ["exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_train_", 
                 "exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_valid_"]

In [ ]:
the_number_of_topic_json_file_df = make_topic_json_txt_file_path_list(json_folder_list, topic_name_list, txt_path_list)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_01 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_01, the_number_of_train_txt_file_list_01 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_01, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_02 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_02, the_number_of_train_txt_file_list_02 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_02, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name +  "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_03 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_03, the_number_of_train_txt_file_list_03 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_03, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_04 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_04, the_number_of_train_txt_file_list_04 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_04, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_05 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_05, the_number_of_train_txt_file_list_05 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_05, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_06 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_06, the_number_of_train_txt_file_list_06 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_06, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_07 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_07, the_number_of_train_txt_file_list_07 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_07, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_08 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_08, the_number_of_train_txt_file_list_08 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_08, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_09 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_09, the_number_of_train_txt_file_list_09 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_09, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_10 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_10, the_number_of_train_txt_file_list_10 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_10, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_11 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_11, the_number_of_train_txt_file_list_11 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_11, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_12 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_12, the_number_of_train_txt_file_list_12 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_12, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_13 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_13, the_number_of_train_txt_file_list_13 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_13, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_14 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_14, the_number_of_train_txt_file_list_14 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_14, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_15 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_15, the_number_of_train_txt_file_list_15 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_15, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_16 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_16, the_number_of_train_txt_file_list_16 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_16, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_17 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_train_txt_file_17, the_number_of_train_txt_file_list_17 = count_number_of_txt_file_with_batch_list(train_json_file_path_list_17, batch_size, corpus_name)

In [ ]:
source_file_by_batch_train_01_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_01.xlsx', engine='openpyxl')  
source_file_by_batch_train_02_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_02.xlsx', engine='openpyxl')  
source_file_by_batch_train_03_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_03.xlsx', engine='openpyxl')  
source_file_by_batch_train_04_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_04.xlsx', engine='openpyxl')  
source_file_by_batch_train_05_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_05.xlsx', engine='openpyxl')  
source_file_by_batch_train_06_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_06.xlsx', engine='openpyxl')  
source_file_by_batch_train_07_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_07.xlsx', engine='openpyxl')  
source_file_by_batch_train_08_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_08.xlsx', engine='openpyxl')  
source_file_by_batch_train_09_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_09.xlsx', engine='openpyxl')  
source_file_by_batch_train_10_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_10.xlsx', engine='openpyxl')  
source_file_by_batch_train_11_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_11.xlsx', engine='openpyxl')  
source_file_by_batch_train_12_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_12.xlsx', engine='openpyxl')  
source_file_by_batch_train_10_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_10.xlsx', engine='openpyxl')  
source_file_by_batch_train_13_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_13.xlsx', engine='openpyxl')  
source_file_by_batch_train_10_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_10.xlsx', engine='openpyxl')  
source_file_by_batch_train_14_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_14.xlsx', engine='openpyxl')  
source_file_by_batch_train_15_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_15.xlsx', engine='openpyxl')  
source_file_by_batch_train_16_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_16.xlsx', engine='openpyxl')  
source_file_by_batch_train_17_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_train_17.xlsx', engine='openpyxl')  

In [ ]:
source_file_by_batch_train_01_df

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_01 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_01, the_number_of_valid_txt_file_list_01 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_01, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_02 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_02, the_number_of_valid_txt_file_list_02 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_02, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_03 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_03, the_number_of_valid_txt_file_list_03 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_03, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_04 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_04, the_number_of_valid_txt_file_list_04 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_04, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_05 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_05, the_number_of_valid_txt_file_list_05 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_05, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_06 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_06, the_number_of_valid_txt_file_list_06 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_06, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_07 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_07, the_number_of_valid_txt_file_list_07 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_07, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_08 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_08, the_number_of_valid_txt_file_list_08 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_08, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_09 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_09, the_number_of_valid_txt_file_list_09 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_09, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_10 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_10, the_number_of_valid_txt_file_list_10 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_10, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_11 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_11, the_number_of_valid_txt_file_list_11 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_11, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_12 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_12, the_number_of_valid_txt_file_list_12 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_12, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_13 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_13, the_number_of_valid_txt_file_list_13 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_13, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_14 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_14, the_number_of_valid_txt_file_list_14 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_14, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_15 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_15, the_number_of_valid_txt_file_list_15 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_15, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_16 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_16, the_number_of_valid_txt_file_list_16 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_16, batch_size, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_17 = source_file_index_df['source_file_path'].values.tolist()
the_number_of_valid_txt_file_17, the_number_of_valid_txt_file_list_17 = count_number_of_txt_file_with_batch_list(valid_json_file_path_list_17, batch_size, corpus_name)

In [ ]:
source_file_by_batch_valid_01_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_01.xlsx', engine='openpyxl')  
source_file_by_batch_valid_02_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_02.xlsx', engine='openpyxl')  
source_file_by_batch_valid_03_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_03.xlsx', engine='openpyxl')  
source_file_by_batch_valid_04_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_04.xlsx', engine='openpyxl')  
source_file_by_batch_valid_05_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_05.xlsx', engine='openpyxl')  
source_file_by_batch_valid_06_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_06.xlsx', engine='openpyxl')  
source_file_by_batch_valid_07_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_07.xlsx', engine='openpyxl')  
source_file_by_batch_valid_08_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_08.xlsx', engine='openpyxl')  
source_file_by_batch_valid_09_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_09.xlsx', engine='openpyxl')  
source_file_by_batch_valid_10_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_10.xlsx', engine='openpyxl')  
source_file_by_batch_valid_11_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_11.xlsx', engine='openpyxl')  
source_file_by_batch_valid_12_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_12.xlsx', engine='openpyxl')  
source_file_by_batch_valid_10_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_10.xlsx', engine='openpyxl')  
source_file_by_batch_valid_13_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_13.xlsx', engine='openpyxl')  
source_file_by_batch_valid_10_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_10.xlsx', engine='openpyxl')  
source_file_by_batch_valid_14_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_14.xlsx', engine='openpyxl')  
source_file_by_batch_valid_15_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_15.xlsx', engine='openpyxl')  
source_file_by_batch_valid_16_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_16.xlsx', engine='openpyxl')  
source_file_by_batch_valid_17_df = pd.read_excel('source_file_by_batch/web_data_based_korean_corpus_data_valid_17.xlsx', engine='openpyxl')  

In [ ]:
source_file_by_batch_valid_01_df

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_01 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_01 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_01, train_txt_file_path_list_01, batch_size, the_number_of_train_txt_file_list_01, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_02 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_02 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_02, train_txt_file_path_list_02, batch_size, the_number_of_train_txt_file_list_02, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_03 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_03 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_03, train_txt_file_path_list_03, batch_size, the_number_of_train_txt_file_list_03, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_04 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_04 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_04, train_txt_file_path_list_04, batch_size, the_number_of_train_txt_file_list_04, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_05 = source_file_index_df['source_file_name'].values.tolist()
train_txt_file_path_list_05 = source_file_index_df['txt_file_name'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_05, train_txt_file_path_list_05, batch_size, the_number_of_train_txt_file_list_05, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_06 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_06 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_06, train_txt_file_path_list_06, batch_size, the_number_of_train_txt_file_list_06, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_07 = source_file_index_df['source_file_name'].values.tolist()
train_txt_file_path_list_07 = source_file_index_df['txt_file_name'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_07, train_txt_file_path_list_07, batch_size, the_number_of_train_txt_file_list_07, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_08 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_08 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_08, train_txt_file_path_list_08, batch_size, the_number_of_train_txt_file_list_08, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_09 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_09 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_09, train_txt_file_path_list_09, batch_size, the_number_of_train_txt_file_list_09, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_10 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_10 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_10, train_txt_file_path_list_10, batch_size, the_number_of_train_txt_file_list_10, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_11 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_11 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_11, train_txt_file_path_list_11, batch_size, the_number_of_train_txt_file_list_11, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_12 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_12 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_12, train_txt_file_path_list_12, batch_size, the_number_of_train_txt_file_list_12, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_13 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_13 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_13, train_txt_file_path_list_13, batch_size, the_number_of_train_txt_file_list_13, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_14 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_14 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_14, train_txt_file_path_list_14, batch_size, the_number_of_train_txt_file_list_14, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_15 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_15 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_15, train_txt_file_path_list_15, batch_size, the_number_of_train_txt_file_list_15, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_16 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_16 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_16, train_txt_file_path_list_16, batch_size, the_number_of_train_txt_file_list_16, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_train" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
train_json_file_path_list_17 = source_file_index_df['source_file_path'].values.tolist()
train_txt_file_path_list_17 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(train_json_file_path_list_17, train_txt_file_path_list_17, batch_size, the_number_of_train_txt_file_list_17, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_01 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_01 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_01, valid_txt_file_path_list_01, batch_size, the_number_of_valid_txt_file_list_01, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_02 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_02 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_02, valid_txt_file_path_list_02, batch_size, the_number_of_valid_txt_file_list_02, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_03 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_03 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_03, valid_txt_file_path_list_03, batch_size, the_number_of_valid_txt_file_list_03, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_04 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_04 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_04, valid_txt_file_path_list_04, batch_size, the_number_of_valid_txt_file_list_04, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_05 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_05 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_05, valid_txt_file_path_list_05, batch_size, the_number_of_valid_txt_file_list_05, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_06 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_06 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_06, valid_txt_file_path_list_06, batch_size, the_number_of_valid_txt_file_list_06, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_07 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_07 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_07, valid_txt_file_path_list_07, batch_size, the_number_of_valid_txt_file_list_07, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_08 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_08 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_08, valid_txt_file_path_list_08, batch_size, the_number_of_valid_txt_file_list_08, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_09 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_09 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_09, valid_txt_file_path_list_09, batch_size, the_number_of_valid_txt_file_list_09, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_10 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_10 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_10, valid_txt_file_path_list_10, batch_size, the_number_of_valid_txt_file_list_10, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_11 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_11 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_11, valid_txt_file_path_list_11, batch_size, the_number_of_valid_txt_file_list_11, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_12 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_12 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_12, valid_txt_file_path_list_12, batch_size, the_number_of_valid_txt_file_list_12, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_13 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_13 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_13, valid_txt_file_path_list_13, batch_size, the_number_of_valid_txt_file_list_13, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_14 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_14 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_14, valid_txt_file_path_list_14, batch_size, the_number_of_valid_txt_file_list_14, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_15 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_15 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_15, valid_txt_file_path_list_15, batch_size, the_number_of_valid_txt_file_list_15, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_16 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_16 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_16, valid_txt_file_path_list_16, batch_size, the_number_of_valid_txt_file_list_16, corpus_name)

In [ ]:
batch_size = 1000
corpus_name = "web_data_based_korean_corpus_data"
source_file_index_xlsx_path = "source_file_index/" + corpus_name + "_valid" + "_source_file_index.xlsx"
source_file_index_df = pd.read_excel(source_file_index_xlsx_path, engine='openpyxl')
valid_json_file_path_list_17 = source_file_index_df['source_file_path'].values.tolist()
valid_txt_file_path_list_17 = source_file_index_df['txt_file_path'].values.tolist()
write_jsontext_to_txt_file_with_batch_list(valid_json_file_path_list_17, valid_txt_file_path_list_17, batch_size, the_number_of_valid_txt_file_list_17, corpus_name)

#### Preprocess TXT File

In [ ]:
from sentence_segmentation import preprocessing_text
from data_preprocessing import make_pro_post_txt_file_path_list
from data_preprocessing import make_topic_pro_post_txt_file_name_list
from data_preprocessing import merge_and_deduplicate_topic_corpus_txt
from reading_data import reading_txt

In [ ]:
pro_corpus_name = "AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_"

topic_name_list = ['01_IT_과학', '02_건강', '03_경제', '04_교육', '05_국제', '06_라이프스타일', '07_문화',
                  '08_사건사고', '09_사회일반', '10_산업', '11_스포츠', '12_여성복지', '13_여행레저',
                  '14_연예', '15_정치', '16_지역', '17_취미']

make_topic_pro_post_txt_file_name_list(pro_corpus_name, topic_name_list)

In [ ]:
pro_post_xlsx_path = "pro_post_corpus_path/web_data_based_korean_corpus_data_" + "train_" + "01" + ".xlsx"
pro_post_total_corpus_path_df = pd.read_excel(pro_post_xlsx_path, engine='openpyxl')
pro_total_corpus_path_list = pro_post_total_corpus_path_df['Pro'].values.tolist()
post_total_corpus_path_list = pro_post_total_corpus_path_df['Post'].values.tolist()

In [ ]:
len(pro_total_corpus_path_list)

In [ ]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "source"

reading_txt(pro_coprus_file, line_length, data_type)

In [ ]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "preprocessing"

reading_txt(pro_coprus_file, line_length, data_type)

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '01' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '02' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '03' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '04' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '05' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '06' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '07' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '08' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '09' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '10' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '11' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '12' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '13' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '14' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '15' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '16' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'train_' + '17' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '01' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '02' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '03' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '04' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '05' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '06' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '07' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '08' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '09' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '10' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '11' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '12' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '13' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '14' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '15' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '16' + '.xlsx' \
    --start_index 0 

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/web_data_based_korean_corpus_data_' + 'valid_' + '17' + '.xlsx' \
    --start_index 0 

In [ ]:
ray.shutdown()

In [ ]:
preprocessing_corpus_path = "AIHUB_corpus/exploration/web_data_based_korean_corpus_data_post/AIHUB_web_data_based_korean_corpus_data_" +"*.txt"
merge_corpus_path = 'AIHUB_corpus/duplicate/AIHUB_web_data_based_korean_corpus_data_'
deduplicate_corpus_path = 'AIHUB_corpus/AIHUB_web_data_based_korean_corpus_data.txt'

topic_name_list = ['01_IT_과학', '02_건강', '03_경제', '04_교육', '05_국제', '06_라이프스타일',
                   '07_문화', '08_사건사고', '09_사회일반', '10_산업', '11_스포츠', '12_여성복지',
                   '13_여행레저', '14_연예', '15_정치', '16_지역', '17_취미']

In [ ]:
merge_and_deduplicate_topic_corpus_txt(preprocessing_corpus_path, merge_corpus_path,
                                        deduplicate_corpus_path, topic_name_list)